In [1]:
def parse(lst):
    it = iter(lst)
    title = next(it)
    columns = next(it).split()[1:]
    xy = [i.split()[1:] for i in it]
    return title, columns, xy

def modify(lst, transpose=True):
    out = []
    for row in lst:
        out.append(this:=[])
        for val in row:
            match val:
                case '-': this.append(None)
                case str() if val.endswith('%'): this.append(float(val[:-1]))
                case str(): this.append(int(val))
    return list(zip(*out)) if transpose else out

data = [
    [i.strip() for i in each.split('\r\n') if i.strip()]
    for each in open("data/2.txt", "rb").read().decode().split('####')
]

from pyecharts.charts import *
from pyecharts import options as opts
import main; all_plots = main.all_plots
global_theme = main.global_theme

def show_line(i, indices, x_name, y_name, legend=1):
    title, columns, xy = parse(data[i])
    yx = modify(xy)
    line = (
        Line(main.get_init_options())
        .add_xaxis("2015 2016 2017 2018 2019".split())
        .set_global_opts(
            xaxis_opts=opts.AxisOpts(name=x_name),
            yaxis_opts=opts.AxisOpts(name=y_name, is_scale=True),
        )
    )
    match legend:
        case 1:
            for i in indices:
                line.add_yaxis(
                    "", yx[i], is_connect_nones=True, is_smooth=False,
                    linestyle_opts=opts.LineStyleOpts(width=3.5),
                    # itemstyle_opts=opts.ItemStyleOpts(border_width=2.75),
                    symbol_size=13, symbol=['circle', 'rect', 'roundRect', 'triangle', 'diamond', 'pin', 'arrow'][i%7]
                )
        case 2:
            for i in indices:
                line.add_yaxis("", yx[i], is_connect_nones=True, linestyle_opts=opts.LineStyleOpts(width=2), symbol_size=5)
    all_plots.append(line)
    return line.render_notebook()

def show_two_side(i, x_name, y_name):
    title, columns, xy = parse(data[i])
    yx = modify(xy)
    y1 = yx[0]
    y2 = [sum(j for j in i[1:] if j is not None) / (len(columns)-1-i[1:].count(None)) for i in zip(*yx)]
    line = (
        Line(main.get_init_options())
        .add_xaxis("2015 2016 2017 2018 2019".split())
        .set_global_opts(
            xaxis_opts=opts.AxisOpts(name=x_name),
            yaxis_opts=opts.AxisOpts(name=y_name, is_scale=True)
        )
        .add_yaxis(
            "", y1, is_connect_nones=True,
            linestyle_opts=opts.LineStyleOpts(width=3.5),
            itemstyle_opts=opts.ItemStyleOpts(border_width=2.75),
            symbol_size=7.5
        )
        .add_yaxis(
            "", [f"{i:.2f}" for i in y2], is_connect_nones=True,
            linestyle_opts=opts.LineStyleOpts(width=3.5),
            itemstyle_opts=opts.ItemStyleOpts(border_width=2.75),
            symbol_size=7.5
        )
    )
    all_plots.append(line)
    return line.render_notebook()

def show_bar(i, indices, x_name, y_name):
    title, columns, xy = parse(data[i])
    yx = modify(xy)
    bar = (
        Bar(main.get_init_options())
        .add_xaxis("2015 2016 2017 2018 2019".split())
        .set_global_opts(
            xaxis_opts=opts.AxisOpts(name=x_name),
            yaxis_opts=opts.AxisOpts(name=y_name, is_scale=True),
            legend_opts=opts.LegendOpts(is_show=False)
        )
    )
    for i in indices:
        bar.add_yaxis("", yx[i])
    all_plots.append(bar)
    return bar.render_notebook()

def show_plots(i, lines=(), bars=(), percentage=False):
    title, columns, xy = parse(data[i])
    yx = modify(xy)
    smooth = False

    line = bar = None

    if lines:
        line = (
            Line(opts.InitOpts("840px", "360px", theme=global_theme))
            .add_xaxis("2015 2016 2017 2018 2019".split())
            .set_global_opts(
                # title_opts=opts.TitleOpts(f"{title}: {subtitle}"),
                yaxis_opts=opts.AxisOpts(name="比例/%" if percentage else "人数/人", is_scale=True),
                xaxis_opts=opts.AxisOpts(name="年份"),
                legend_opts=opts.LegendOpts(
                    pos_right=0, pos_bottom="15%", align="right", orient="vertical"
                )
            )
        )
        for i in lines:
            line.add_yaxis(
                columns[i], yx[i], is_connect_nones=True, is_smooth=smooth,
                linestyle_opts=opts.LineStyleOpts(width=2.5),
                itemstyle_opts=opts.ItemStyleOpts(border_width=1.5)
            )
        all_plots.append(line)
    if bars:
        bar = (
            Bar(opts.InitOpts("920px", "480px", theme=global_theme))
            .add_xaxis("2015 2016 2017 2018 2019".split())
            .set_global_opts(
                # title_opts=opts.TitleOpts(f"{title}: {subtitle}"),
                yaxis_opts=opts.AxisOpts(name="比例/%", is_scale=True),
                xaxis_opts=opts.AxisOpts(name="年份"),
                legend_opts=opts.LegendOpts(
                    pos_right=0, pos_bottom="15%", align="right", orient="vertical"
                )
            )
        )
        for i in bars:
            bar.add_yaxis(columns[i], yx[i])
        all_plots.append(bar)

    return (Page(title).add(line,bar) if line and bar else line or bar).render_notebook()

In [2]:
# show_plots(0, (0,1), (2,3))
show_bar(0, (2,3), "年份", "比例")

In [3]:
# show_plots(1, range(len(data[1][1].split())-1), percentage=True)
show_line(1, range(len(data[1][1].split())-1), "年份", "人数")

In [4]:
# show_plots(2, range(len(data[2][1].split())-1), percentage=True)
show_line(2, range(len(data[2][1].split())-1), "年份", "比例")

In [5]:
# show_plots(3, range(len(data[3][1].split())-1), percentage=True)
show_line(3, range(len(data[3][1].split())-1), "年份", "比例")

In [6]:
# show_plots(4, range(len(data[4][1].split())-1), percentage=True)

In [7]:
show_two_side(4, "年份", "比例")

In [8]:
show_line(4, range(1, len(data[4][1].split())-1), "年份", "比例", legend=2)

In [9]:
# show_plots(5, range(len(data[5][1].split())-1), percentage=True)

In [10]:
show_two_side(5, "年份", "比例")

In [11]:
show_line(5, range(1, len(data[5][1].split())-1), "年份", "比例", legend=2)

In [12]:
from main import riverize

def get_xy(index):
    title, columns, xy = parse(data[index])
    return title, columns, modify(xy, False)

def show_river(index):
    title, columns, xy = get_xy(index)
    return riverize(xy, [str(i) for i in range(2015,2020)], columns, title, "年份", 2015, 2019)

In [13]:
show_river(1)

In [14]:
show_river(2)

In [15]:
show_river(3)

In [16]:
main.show_all()